# Test Environment for Date Binning

Addresses issue #8 in the [forecast repo](https://github.com/agritheory/forecast/issues/8):

>Having a date/period binning utility would be really handy for this project. Some high level requirements:
>
>- Arbitrary start and end dates, able to consume or output partial periods
>- Establish patterns around periodicity:
>    - Week with arbitrary weekday start vs ISO week
>    - Calendar month with remainders (EG Jan 15 to Mar 15, binning by calendar month would return a bin with Jan 15-31, Feb 1-28, Mar 1-15)
>    - Various ISO month patterns (4 5 4, 4 4 5, 13 4-week months) - there isn't a specific convention here as far as I know, but several are used in practice.
>- It would be great to be able to convert a set of data from one set of period-based bins to another. This will be particularly handy when converting /distributing summary level calendar month data into ISO weeks, for example
>- It may be appropriate to have a separate abstraction(s)/ function for periodicity, which would allow for more reasonable testing of the binning parameters.
>- This feels like something that's in the itertools/more_itertools family, but with special rules.
>
>There are probably more requirements than just these above, I think it would make sense to start with some example data. We can design that offline to make sure it includes the sorts of test cases we want to hit on.

## Resources

- [Python 3.10 itertools documentation](https://docs.python.org/3.10/library/itertools.html)
- [ISO 8601 Format](https://en.wikipedia.org/wiki/ISO_8601)
- [Python 3.10 datetime documentation](https://docs.python.org/3.10/library/datetime.html)

In [2]:
from itertools import pairwise, cycle
import datetime

import frappe
from frappe.model.document import Document
from fab.forecast_and_budget.state import XStateDocument
from dateutil.relativedelta import relativedelta

In [3]:
class ForecastAndBudgetPeriod(XStateDocument):
    def __init__(self, start_date, end_date, periodicity=None):
        """
        Needed to get class to work in Jupyter - TODO: configure as
            XStateDocument
            
        """
        self.start_date = start_date
        self.end_date = end_date
        self.periodicity = periodicity

    def get_iso_start_dates(self, start_date, end_date, step):
        """
        Returns a list of datetime.date objects representing the starting date of
            all ISO periods falling within the time span defined by `start_date` to
            `end_date`. If `start_date` is not a Monday (the starting weekday for
            an ISO week), then the first bin will begin with a partial week
            (`start_date` to Sunday). All following bins will consist of full
            periods.
        
        :param start_date: datetime.date object; the date to start binning from
        :param end_date: datetime.date object; the date to end binning
        :param step: tuple of str, int or sequence of ints; the string indicates
            the portion of the date that is changing via step (this function assumes
            "weeks"), and the integer or sequence of integers is the step for period
            dates generated within the time span. If `step` is a sequence, it will
            cycle continuously over the values to get the current step to collect
            the start dates of the periods.
            Example: ISO Month (4 + 5 + 4) would use (4, 5, 4) as the `step`, so
            the returned sequence of dates represent months that are 4 ISO weeks,
            then 5 ISO weeks, then 4 ISO weeks, with that pattern continuing through
            to the `end_date`
        :return: list of datetime.date objects
        """
        period, delta = step
        r = []
        
        if type(delta) is int:
            delta = [delta]
        seq = cycle(delta)
        while start_date < end_date:
            r.append(start_date)
            if start_date.isoweekday() != 1:  # Reset to prior Monday before date math
                start_date = datetime.date.fromisocalendar(start_date.isocalendar().year, start_date.isocalendar().week, 1)
            start_date += relativedelta(weeks=next(seq))
        
        return r
    
    def get_iso_annual_start_dates(self, start_date, end_date, step):
        """
        Returns a list of datetime.date objects representing the starting date of
            all ISO years falling within the time span defined by `start_date` to
            `end_date`. If `start_date` is not the first day of that ISO year, the
            returned sequence will begin with a "stub" period (an incomplete year).
            The second date in the returned sequence will be the first day of the
            first full ISO year after `start_date`.
        
        :param start_date: datetime.date object; the date to start binning from
        :param end_date: datetime.date object; the date to end binning
        :param step: tuple of str, int; the string indicates the portion of the date
            that is changing via step, and the integer is the step
        :return: list of datetime.date objects
        """
        period, delta = step
        r = []        
        iso_yr_start_date = datetime.date.fromisocalendar(start_date.year, 1, 1)
        
        # Start_date falls before the first day of that ISO year
        if start_date < iso_yr_start_date:
            r.append(start_date)
            start_date = iso_yr_start_date
        # Start_date falls after the first day of that ISO year
        elif start_date > iso_yr_start_date:
            r.append(start_date)
            start_date = datetime.date.fromisocalendar(start_date.year + delta, 1, 1)
        
        while start_date < end_date:
            r.append(start_date)
            start_date = datetime.date.fromisocalendar(start_date.year + delta, 1, 1)
        
        return r
    
    def get_cal_start_dates(self, start_date, end_date, step):
        """
        Returns a list of datetime.date objects representing the starting date of
            all calendar-based periods falling within the time span defined by
            `start_date` to `end_date`. The step tuple is determined by periodicity
            
            
            If `start_date` is not a Monday (the starting weekday for
            an ISO week), then the returned sequence will begin with a "stub" period
            (an incomplete week). The second date in the returned sequence will be
            the first Monday after `start_date`, and all following dates are Mondays.
        
        :param start_date: datetime.date object; the date to start binning from
        :param end_date: datetime.date object; the date to end binning
        :param step: tuple of str, int; the string indicates the portion of the date
            that is changing via step, and the integer is the step
        :return: list of datetime.date objects        
        """
        period, delta = step
        r = []
        
        if period == "weeks":
            while start_date < end_date:
                r.append(start_date)
                start_date += relativedelta(weeks=delta)
        elif period == "months":
            while start_date < end_date:
                r.append(start_date)
                if start_date.day != 1:
                    start_date = start_date.replace(day=1)  # Reset to first of month before date math
                start_date += relativedelta(months=delta)
        elif period == "years":
            while start_date < end_date:
                r.append(start_date)
                start_date += relativedelta(years=delta)
        elif period == "years-cy":
            while start_date < end_date:
                r.append(start_date)
                if start_date.day != 1 or start_date.month != 1:
                    start_date = start_date.replace(month=1, day=1)  # Reset to first of year before date math
                start_date += relativedelta(years=delta)
        
        return r
    
    def get_period_end_dates(self, start_dates):
        """
        Accepts a sequence of period start dates and adds the period
            end date for each item, which is the day prior to the next
            start date in the sequence. Assumes the final date is the
            (exclusive) end date for the entire sequence.

        :param start_dates: sequence of datetime.date objects
        :return: list of tuples of datetime.date objects; each tuple 
            represents the start and end dates of end-to-end periods
        """
        # Generate (from, to) period pairs off start dates
        return [(p[0], p[1] + relativedelta(days=-1)) for p in pairwise(start_dates)]
    
    def get_dates(self, start_date=None, end_date=None, periodicity=None, inclusive=False):
        """
        Gets the starting dates for all periods falling within the time span
            from `start_date` to `end_date`, then returns a list of tuples
            with the start and end dates for each period.
        
        :param start_date: datetime.date object; the date to start binning from
        :param end_date: datetime.date object; the date to end binning
        :param periodicity: str or None; how to determine the periods within the
            time span from `start_date` to `end_date`. If None, assumes "Weekly".
            Options (for all ISO periods, if `start_date` is not a Monday, the
            first bin will include a partial week (`start_date` to Sunday), with
            following bins generated with full ISO weeks):
            - "Weekly": weekly bins where the starting weekday is determined by
              `start_date`
            - "ISO Week": weekly bins starting on a Monday
            - "Biweekly": bins of 2 week periods, starting on `start_date`'s weekday
            - "ISO Biweekly": bins of 2 ISO week periods
            - "Calendar Month": monthly bins by calendar. If `start_date` is not the
              first of the month, the first bin will be a partial month
            - "ISO Month (4 Weeks)": monthly bins of 4 ISO week periods
            - "ISO Month (4 + 5 + 4)": monthly bins of ISO week periods following a
              pattern where the first month is 4 ISO weeks long, the next is 5 ISO
              weeks long, the next is 4 ISO weeks long, repeating until `end_date`
            - "ISO Month (4 + 4 + 5)": monthly bins of ISO week periods following a
              pattern where the first month is 4 ISO weeks long, the next is 4 ISO
              weeks long, the next is 5 ISO weeks long, repeating until `end_date`
            - "Calendar Quarter": bins of 3 calendar month periods. If `start_date`
              is not the first of the month, the first month of the first bin will
              be a partial month
            - "ISO Quarter (13 Weeks)": quarterly bins of 13 ISO week periods
            - "ISO Semiannual (26 Weeks)": semiannual bins of 26 ISO week periods
            - "Calendar Year": calendar year bins (Jan-Dec). If `start_date`
              is not Jan 1, the first bin will be a partial year
            - "Annually": yearly bins starting from `start_date`
            - "ISO Annual": bins of full ISO years. If `start_date` is not the first
              day of the ISO year, the first bin will be a partial year
            - "Entire Period": one bin from `start_date` to either `end_date` (if
              inclusive=True) or the day prior to `end_date` (if inclusive=False)
        :param inclusive: bool; if the time span being binned includes the end_date
            (inclsive=True) or ends the day before (inclusive=False).
        :return: list of tuples in form `(datetime.date object, datetime.date object)`
        """
        start_date = self.start_date if not start_date else start_date
        end_date = self.end_date if not end_date else end_date
        effective_end_date = end_date if not inclusive else end_date + relativedelta(days=1)
        periodicity = self.periodicity if not periodicity else periodicity
        is_iso = 'iso' in periodicity.lower() if periodicity is not None else False
        
        steps = {
            "ISO Week": ("weeks", 1),
            "ISO Biweekly": ("weeks", 2),
            "ISO Month (4 Weeks)": ("weeks", 4),
            "ISO Month (4 + 5 + 4)": ("weeks", (4, 5, 4)),
            "ISO Month (4 + 4 + 5)": ("weeks", (4, 4, 5)),
            "ISO Quarter (13 Weeks)": ("weeks", 13), 
            "ISO Semiannual (26 Weeks)": ("weeks", 26),
            "ISO Annual": ("years", 1),  # edge case (years can be 52 or 53 weeks)
            "Weekly": ("weeks", 1),
            "Biweekly": ("weeks", 2),
            "Calendar Month": ("months", 1),
            "Calendar Quarter": ("months", 3),
            "Calendar Year": ("years-cy", 1),  # assumes Jan-Dec year (accommodates partial years on front or back end)
            "Annually": ("years", 1),  # assumes 1 year from date given (accommodates non-calendar FYs)            
        }
        
        if periodicity == "Entire Period":
            return [(start_date, effective_end_date)]
        
        step = steps.get(periodicity, steps["Weekly"])
        print(f'{start_date=}\n{end_date=}, {effective_end_date=}\n{step=}')
        if is_iso:
            if periodicity == "ISO Annual":
                r = self.get_iso_annual_start_dates(start_date, effective_end_date, step)
            else:
                r = self.get_iso_start_dates(start_date, effective_end_date, step)
        else:
            r = self.get_cal_start_dates(start_date, effective_end_date, step)

        r.append(effective_end_date)
        r = self.get_period_end_dates(r)
        return r
    
    def period_labels(self, periodicity):
        month_names = ["Jan","Feb","Mar","Apr","May","Jun","Jul","Aug","Sep","Oct","Nov","Dec"]
        if periodicity in ('ISO Week', 'Weekly', None):
            return [p.strftime("%m/%d/%Y") for p in self.get_dates(periodicity=periodicity)]
        if periodicity == "ISO Month (4 + 5 + 4)":
            start_month = self.start_date.month + 1 % 12
            month_sizes = ["4","5","4","4","5","4","4","5","4","4","5","4"]
            month_names = month_names[start_month:] + month_names[:start_month]
            return month_names
        elif periodicity == 'Calendar Month':
            return month_names
        elif periodicity == "ISO Quarter (13 Weeks)":
            # get quarter 
            return ['Q1', 'Q2', 'Q3', 'Q4']  # TODO: what if Q's are back end of their year and don't start at Q1? Use "Q-mm/dd/yy"?
        elif periodicity == 'Entire Period':
            return [self.start_date.strftime("%m/%d/%Y")]


### ISO Tests

**ISO Week**

- Standard: 1/2/23-12/31/23 (inclusive=True)
    - w01: 1/2/23 to 1/8/23
    - w02: 1/9/15 to 1/15/23
    - w03: 1/16/23 to 1/22/23
    - w04: 1/23/23 to 1/29/23
    - w05: 1/30/23 to 2/5/23
    - w06: 2/6/23 to 2/12/23
    - w07: 2/13/23 to 2/19/23
    - w08: 2/20/23 to 2/26/23
    - w09: 2/27/23 to 3/5/23
    - w10: 3/6/23 to 3/12/23
    - w11: 3/13/23 to 3/19/23
    - w12: 3/20/23 to 3/26/23
    - w13: 3/27/23 to 4/2/23
    - w14: 4/3/23 to 4/9/23
    - w15: 4/10/23 to 4/16/23
    - w16: 4/17/23 to 4/23/23
    - w17: 4/24/23 to 4/30/23
    - w18: 5/1/23 to 5/7/23
    - w19: 5/8/23 to 5/14/23
    - w20: 5/15/23 to 5/21/23
    - w21: 5/22/23 to 5/28/23
    - w22: 5/29/23 to 6/4/23
    - w23: 6/5/23 to 6/11/23
    - w24: 6/12/23 to 6/18/23
    - w25: 6/19/23 to 6/25/23
    - w26: 6/26/23 to 7/2/23
    - w27: 7/3/23 to 7/9/23
    - w28: 7/10/23 to 7/16/23
    - w29: 7/17/23 to 7/23/23
    - w30: 7/24/23 to 7/30/23
    - w31: 7/31/23 to 8/6/23
    - w32: 8/7/23 to 8/13/23
    - w33: 8/14/23 to 8/20/23
    - w34: 8/21/23 to 8/27/23
    - w35: 8/28/23 to 9/3/23
    - w36: 9/4/23 to 9/10/23
    - w37: 9/11/12 to 9/17/23
    - w38: 9/18/23 to 9/24/23
    - w39: 9/25/23 to 10/1/23
    - w40: 10/2/23 to 10/8/23
    - w41: 10/9/23 to 10/15/23
    - w42: 10/16/23 to 10/22/23
    - w43: 10/23/23 to 10/29/23
    - w44: 10/30/23 to 11/5/23
    - w45: 11/6/23 to 11/12/23
    - w46: 11/13/23 to 11/19/23
    - w47: 11/20/23 to 11/26/23
    - w48: 11/27/23 to 12/3/23
    - w49: 12/4/23 to 12/10/23
    - w50: 12/11/23 to 12/17/23
    - w51: 12/18/23 to 12/24/23
    - w52: 12/25/23 to 12/31/23
- Stub period: 1/1/23-1/12/23 (inclusive=False)
    - w01: 1/1/23 to 1/1/23
    - w02: 1/2/23 to 1/8/23
    - w02: 1/9/23 to 1/11/23

In [4]:
# ISO Week Test 1
sd = datetime.date(2023, 1, 2)
ed = datetime.date(2023, 12, 31)
ForecastAndBudgetPeriod(sd, ed, "ISO Week").get_dates(inclusive=True)

start_date=datetime.date(2023, 1, 2)
end_date=datetime.date(2023, 12, 31), effective_end_date=datetime.date(2024, 1, 1)
step=('weeks', 1)


[(datetime.date(2023, 1, 2), datetime.date(2023, 1, 8)),
 (datetime.date(2023, 1, 9), datetime.date(2023, 1, 15)),
 (datetime.date(2023, 1, 16), datetime.date(2023, 1, 22)),
 (datetime.date(2023, 1, 23), datetime.date(2023, 1, 29)),
 (datetime.date(2023, 1, 30), datetime.date(2023, 2, 5)),
 (datetime.date(2023, 2, 6), datetime.date(2023, 2, 12)),
 (datetime.date(2023, 2, 13), datetime.date(2023, 2, 19)),
 (datetime.date(2023, 2, 20), datetime.date(2023, 2, 26)),
 (datetime.date(2023, 2, 27), datetime.date(2023, 3, 5)),
 (datetime.date(2023, 3, 6), datetime.date(2023, 3, 12)),
 (datetime.date(2023, 3, 13), datetime.date(2023, 3, 19)),
 (datetime.date(2023, 3, 20), datetime.date(2023, 3, 26)),
 (datetime.date(2023, 3, 27), datetime.date(2023, 4, 2)),
 (datetime.date(2023, 4, 3), datetime.date(2023, 4, 9)),
 (datetime.date(2023, 4, 10), datetime.date(2023, 4, 16)),
 (datetime.date(2023, 4, 17), datetime.date(2023, 4, 23)),
 (datetime.date(2023, 4, 24), datetime.date(2023, 4, 30)),
 (datet

In [5]:
# ISO Week Test 2
sd = datetime.date(2023, 1, 1)
ed = datetime.date(2023, 1, 12)
ForecastAndBudgetPeriod(sd, ed, "ISO Week").get_dates(inclusive=False)

start_date=datetime.date(2023, 1, 1)
end_date=datetime.date(2023, 1, 12), effective_end_date=datetime.date(2023, 1, 12)
step=('weeks', 1)


[(datetime.date(2023, 1, 1), datetime.date(2023, 1, 1)),
 (datetime.date(2023, 1, 2), datetime.date(2023, 1, 8)),
 (datetime.date(2023, 1, 9), datetime.date(2023, 1, 11))]

**ISO Month (4 + 5 + 4)**:

- 1/2/23-12/31/23 (inclusive=True)
    - m01: 1/2/23 to 1/29/23
    - m02: 1/30/23 to 3/5/23
    - m03: 3/6/23 to 4/2/23
    - m04: 4/3/23 to 4/30/23
    - m05: 5/1/23 to 6/4/23
    - m06: 6/5/23 to 7/2/23
    - m07: 7/3/23 to 7/30/23
    - m08: 7/31/23 to 9/3/23
    - m09: 9/4/23 to 10/1/23
    - m10: 10/2/23 to 10/29/23
    - m11: 10/30/23 12/3/23
    - m12: 12/4/23 to 12/31/23

In [6]:
# ISO Month (4 + 5 + 4) Test
sd = datetime.date(2023, 1, 2)
ed = datetime.date(2023, 12, 31)
ForecastAndBudgetPeriod(sd, ed, "ISO Month (4 + 5 + 4)").get_dates(inclusive=True)

start_date=datetime.date(2023, 1, 2)
end_date=datetime.date(2023, 12, 31), effective_end_date=datetime.date(2024, 1, 1)
step=('weeks', (4, 5, 4))


[(datetime.date(2023, 1, 2), datetime.date(2023, 1, 29)),
 (datetime.date(2023, 1, 30), datetime.date(2023, 3, 5)),
 (datetime.date(2023, 3, 6), datetime.date(2023, 4, 2)),
 (datetime.date(2023, 4, 3), datetime.date(2023, 4, 30)),
 (datetime.date(2023, 5, 1), datetime.date(2023, 6, 4)),
 (datetime.date(2023, 6, 5), datetime.date(2023, 7, 2)),
 (datetime.date(2023, 7, 3), datetime.date(2023, 7, 30)),
 (datetime.date(2023, 7, 31), datetime.date(2023, 9, 3)),
 (datetime.date(2023, 9, 4), datetime.date(2023, 10, 1)),
 (datetime.date(2023, 10, 2), datetime.date(2023, 10, 29)),
 (datetime.date(2023, 10, 30), datetime.date(2023, 12, 3)),
 (datetime.date(2023, 12, 4), datetime.date(2023, 12, 31))]

**ISO Month (4 + 4 + 5)**:

- 1/2/23-12/31/23 (inclusive=True)
    - m01: 1/2/23 to 1/29/23
    - m02: 1/30/23 to 2/26/23
    - m03: 2/27/23 to 4/2/23
    - m04: 4/3/23 to 4/30/23
    - m05: 5/1/23 to 5/28/23
    - m06: 5/29/23 to 7/2/23
    - m07: 7/3/23 to 7/30/23
    - m08: 7/31/23 to 8/27/23
    - m09: 8/28/23 to 10/1/23
    - m10: 10/2/23 to 10/29/23
    - m11: 10/30/23 to 11/26/23
    - m12: 11/27/23 to 12/31/23

In [7]:
# ISO Month (4 + 4 + 5) Test
sd = datetime.date(2023, 1, 2)
ed = datetime.date(2023, 12, 31)
ForecastAndBudgetPeriod(sd, ed, "ISO Month (4 + 4 + 5)").get_dates(inclusive=True)

start_date=datetime.date(2023, 1, 2)
end_date=datetime.date(2023, 12, 31), effective_end_date=datetime.date(2024, 1, 1)
step=('weeks', (4, 4, 5))


[(datetime.date(2023, 1, 2), datetime.date(2023, 1, 29)),
 (datetime.date(2023, 1, 30), datetime.date(2023, 2, 26)),
 (datetime.date(2023, 2, 27), datetime.date(2023, 4, 2)),
 (datetime.date(2023, 4, 3), datetime.date(2023, 4, 30)),
 (datetime.date(2023, 5, 1), datetime.date(2023, 5, 28)),
 (datetime.date(2023, 5, 29), datetime.date(2023, 7, 2)),
 (datetime.date(2023, 7, 3), datetime.date(2023, 7, 30)),
 (datetime.date(2023, 7, 31), datetime.date(2023, 8, 27)),
 (datetime.date(2023, 8, 28), datetime.date(2023, 10, 1)),
 (datetime.date(2023, 10, 2), datetime.date(2023, 10, 29)),
 (datetime.date(2023, 10, 30), datetime.date(2023, 11, 26)),
 (datetime.date(2023, 11, 27), datetime.date(2023, 12, 31))]

**ISO Month (4 Weeks)**:

- Standard: 1/2/23-12/31/23 (inclusive=True)
    - m01: 1/2/23 to 1/29/23
    - m02: 1/30/23 to 2/26/23
    - m03: 2/27/23 to 3/26/23
    - m04: 3/27/23 to 4/23/23
    - m05: 4/24/23 to 5/21/23
    - m06: 5/22/23 to 6/18/23
    - m07: 6/19/23 to 7/16/23
    - m08: 7/17/23 to 8/13/23
    - m09: 8/14/23 to 9/10/23
    - m10: 9/11/12 to 10/8/23
    - m11: 10/9/23 to 11/5/23
    - m12: 11/6/23 to 12/3/23
    - m13: 12/4/23 to 12/31/23
- Stub: 1/5/23-3/26/23 (inclusive=True)
    - m01: 1/5/23 to 1/29/23
    - m02: 1/30/23 to 2/26/23
    - m03: 2/27/23 to 3/26/23

In [8]:
# ISO Month (4 Weeks) Test 1
sd = datetime.date(2023, 1, 2)
ed = datetime.date(2023, 12, 31)
ForecastAndBudgetPeriod(sd, ed, "ISO Month (4 Weeks)").get_dates(inclusive=True)

start_date=datetime.date(2023, 1, 2)
end_date=datetime.date(2023, 12, 31), effective_end_date=datetime.date(2024, 1, 1)
step=('weeks', 4)


[(datetime.date(2023, 1, 2), datetime.date(2023, 1, 29)),
 (datetime.date(2023, 1, 30), datetime.date(2023, 2, 26)),
 (datetime.date(2023, 2, 27), datetime.date(2023, 3, 26)),
 (datetime.date(2023, 3, 27), datetime.date(2023, 4, 23)),
 (datetime.date(2023, 4, 24), datetime.date(2023, 5, 21)),
 (datetime.date(2023, 5, 22), datetime.date(2023, 6, 18)),
 (datetime.date(2023, 6, 19), datetime.date(2023, 7, 16)),
 (datetime.date(2023, 7, 17), datetime.date(2023, 8, 13)),
 (datetime.date(2023, 8, 14), datetime.date(2023, 9, 10)),
 (datetime.date(2023, 9, 11), datetime.date(2023, 10, 8)),
 (datetime.date(2023, 10, 9), datetime.date(2023, 11, 5)),
 (datetime.date(2023, 11, 6), datetime.date(2023, 12, 3)),
 (datetime.date(2023, 12, 4), datetime.date(2023, 12, 31))]

In [9]:
# ISO Month (4 Weeks) Test 2
sd = datetime.date(2023, 1, 5)
ed = datetime.date(2023, 3, 26)
ForecastAndBudgetPeriod(sd, ed, "ISO Month (4 Weeks)").get_dates(inclusive=True)

start_date=datetime.date(2023, 1, 5)
end_date=datetime.date(2023, 3, 26), effective_end_date=datetime.date(2023, 3, 27)
step=('weeks', 4)


[(datetime.date(2023, 1, 5), datetime.date(2023, 1, 29)),
 (datetime.date(2023, 1, 30), datetime.date(2023, 2, 26)),
 (datetime.date(2023, 2, 27), datetime.date(2023, 3, 26))]

**ISO Quarter (13 Weeks)**:

- Standard: 1/2/23-12/31/23 (inclusive=True)
    - Q1: 1/2/23 to 4/2/23
    - Q2: 4/3/23 to 7/2/23
    - Q3: 7/3/23 to 10/1/23
    - Q4: 10/2/23 to 12/31/23
- Stub: 1/5/23-12/31/23 (inclusive=True)
    - Q1: 1/5/23 to 4/2/23
    - Q2: 4/3/23 to 7/2/23
    - Q3: 7/3/23 to 10/1/23
    - Q4: 10/2/23 to 12/31/23

In [10]:
# ISO Quarter Test 1
sd = datetime.date(2023, 1, 2)
ed = datetime.date(2023, 12, 31)
ForecastAndBudgetPeriod(sd, ed, "ISO Quarter (13 Weeks)").get_dates(inclusive=True)

start_date=datetime.date(2023, 1, 2)
end_date=datetime.date(2023, 12, 31), effective_end_date=datetime.date(2024, 1, 1)
step=('weeks', 13)


[(datetime.date(2023, 1, 2), datetime.date(2023, 4, 2)),
 (datetime.date(2023, 4, 3), datetime.date(2023, 7, 2)),
 (datetime.date(2023, 7, 3), datetime.date(2023, 10, 1)),
 (datetime.date(2023, 10, 2), datetime.date(2023, 12, 31))]

In [11]:
# ISO Quarter Test 2
sd = datetime.date(2023, 1, 5)
ed = datetime.date(2023, 12, 31)
ForecastAndBudgetPeriod(sd, ed, "ISO Quarter (13 Weeks)").get_dates(inclusive=True)

start_date=datetime.date(2023, 1, 5)
end_date=datetime.date(2023, 12, 31), effective_end_date=datetime.date(2024, 1, 1)
step=('weeks', 13)


[(datetime.date(2023, 1, 5), datetime.date(2023, 4, 2)),
 (datetime.date(2023, 4, 3), datetime.date(2023, 7, 2)),
 (datetime.date(2023, 7, 3), datetime.date(2023, 10, 1)),
 (datetime.date(2023, 10, 2), datetime.date(2023, 12, 31))]

**ISO Annual**:
- Start date falls on beginning of ISO year: 1/2/23-12/30/24 (inclusive=False) => `[(1/2/23, 12/31/23), (1/1/24, 12/29/24)]`
- Start date falls before ISO year: 1/1/23-12/29/24 (inclusive=True) => `[(1/1/23, 1/1/23), (1/2/23, 12/31/23), (1/1/24, 12/29/24)]`
- Start date falls after ISO year: 7/3/23-12/29/24 (inclusive=True) => `[(7/3/23, 12/31/23), (1/1/24, 12/29/24)]`
- End date falls before ISO year ends: 1/2/23-6/30/24 (inclusive=False) => `[(1/2/23, 12/31/23), (1/1/24, 6/29/24)]`

In [12]:
# ISO Annual Test 1
sd = datetime.date(2023, 1, 2)
ed = datetime.date(2024, 12, 30)
ForecastAndBudgetPeriod(sd, ed, "ISO Annual").get_dates(inclusive=False)

start_date=datetime.date(2023, 1, 2)
end_date=datetime.date(2024, 12, 30), effective_end_date=datetime.date(2024, 12, 30)
step=('years', 1)


[(datetime.date(2023, 1, 2), datetime.date(2023, 12, 31)),
 (datetime.date(2024, 1, 1), datetime.date(2024, 12, 29))]

In [13]:
# ISO Annual Test 2
sd = datetime.date(2023, 1, 1)
ed = datetime.date(2024, 12, 29)
ForecastAndBudgetPeriod(sd, ed, "ISO Annual").get_dates(inclusive=True)

start_date=datetime.date(2023, 1, 1)
end_date=datetime.date(2024, 12, 29), effective_end_date=datetime.date(2024, 12, 30)
step=('years', 1)


[(datetime.date(2023, 1, 1), datetime.date(2023, 1, 1)),
 (datetime.date(2023, 1, 2), datetime.date(2023, 12, 31)),
 (datetime.date(2024, 1, 1), datetime.date(2024, 12, 29))]

In [14]:
# ISO Annual Test 3
sd = datetime.date(2023, 7, 3)
ed = datetime.date(2024, 12, 29)
ForecastAndBudgetPeriod(sd, ed, "ISO Annual").get_dates(inclusive=True)

start_date=datetime.date(2023, 7, 3)
end_date=datetime.date(2024, 12, 29), effective_end_date=datetime.date(2024, 12, 30)
step=('years', 1)


[(datetime.date(2023, 7, 3), datetime.date(2023, 12, 31)),
 (datetime.date(2024, 1, 1), datetime.date(2024, 12, 29))]

In [15]:
# ISO Annual Test 4
sd = datetime.date(2023, 1, 2)
ed = datetime.date(2024, 6, 30)
ForecastAndBudgetPeriod(sd, ed, "ISO Annual").get_dates(inclusive=False)

start_date=datetime.date(2023, 1, 2)
end_date=datetime.date(2024, 6, 30), effective_end_date=datetime.date(2024, 6, 30)
step=('years', 1)


[(datetime.date(2023, 1, 2), datetime.date(2023, 12, 31)),
 (datetime.date(2024, 1, 1), datetime.date(2024, 6, 29))]

### Calendar Date Tests

**Weekly and Biweekly**
- Standard: 1/2/23-2/13/23 (inclusive=False) => see first 6 ISO weeks
- Non-Monday start: 1/5/23-1/26/23 (inclusive=False) => `[(1/5/23, 1/11/23), (1/12/23, 1/18/23), (1/19/23, 1/25/23)]`
- Biweekly: 1/11/12-2/15/23 (inclusive=True) => `[(1/11/23, 1/24/23), (1/25/23, 2/7/23), (2/8/23, 2/15/23)]`

In [16]:
# Weekly Test 1
sd = datetime.date(2023, 1, 2)
ed = datetime.date(2023, 2, 13)
ForecastAndBudgetPeriod(sd, ed, "Weekly").get_dates(inclusive=False)

start_date=datetime.date(2023, 1, 2)
end_date=datetime.date(2023, 2, 13), effective_end_date=datetime.date(2023, 2, 13)
step=('weeks', 1)


[(datetime.date(2023, 1, 2), datetime.date(2023, 1, 8)),
 (datetime.date(2023, 1, 9), datetime.date(2023, 1, 15)),
 (datetime.date(2023, 1, 16), datetime.date(2023, 1, 22)),
 (datetime.date(2023, 1, 23), datetime.date(2023, 1, 29)),
 (datetime.date(2023, 1, 30), datetime.date(2023, 2, 5)),
 (datetime.date(2023, 2, 6), datetime.date(2023, 2, 12))]

In [17]:
# Weekly Test 2
sd = datetime.date(2023, 1, 5)
ed = datetime.date(2023, 1, 26)
ForecastAndBudgetPeriod(sd, ed, "Weekly").get_dates(inclusive=False)

start_date=datetime.date(2023, 1, 5)
end_date=datetime.date(2023, 1, 26), effective_end_date=datetime.date(2023, 1, 26)
step=('weeks', 1)


[(datetime.date(2023, 1, 5), datetime.date(2023, 1, 11)),
 (datetime.date(2023, 1, 12), datetime.date(2023, 1, 18)),
 (datetime.date(2023, 1, 19), datetime.date(2023, 1, 25))]

In [18]:
# Biweekly Test
sd = datetime.date(2023, 1, 11)
ed = datetime.date(2023, 2, 15)
ForecastAndBudgetPeriod(sd, ed, "Biweekly").get_dates(inclusive=True)

start_date=datetime.date(2023, 1, 11)
end_date=datetime.date(2023, 2, 15), effective_end_date=datetime.date(2023, 2, 16)
step=('weeks', 2)


[(datetime.date(2023, 1, 11), datetime.date(2023, 1, 24)),
 (datetime.date(2023, 1, 25), datetime.date(2023, 2, 7)),
 (datetime.date(2023, 2, 8), datetime.date(2023, 2, 15))]

**Calendar Month and Calendar Quarter**

- Calendar Month: 1/15/23-3/15/23 (inclusive=True) => `[(1/15/23, 1/31/23), (2/1/23, 2/28/23), (3/1/23, 3/15/23)]`
- Calendar Quarter: 1/1/23-9/30/23 (inclusive=True) => `[(1/1/23, 3/31/23), (4/1/23, 6/30/23), (7/1/23, 9/30/23)]`
- Calendar Quarter (with stub): 1/10/23-8/10/23 (inclusive=True) => `[(1/10/23, 3/31/23), (4/1/23, 6/30/23), (7/1/23, 8/10/23)]`
- Calendar Quarter (Oct FY): 11/1/23-4/30/24 (inclusive=True) => `[(11/1/23, 1/31/24), (2/1/24, 4/30/24)]`

In [19]:
# Calendar Month Test
sd = datetime.date(2023, 1, 15)
ed = datetime.date(2023, 3, 15)
ForecastAndBudgetPeriod(sd, ed, "Calendar Month").get_dates(inclusive=True)

start_date=datetime.date(2023, 1, 15)
end_date=datetime.date(2023, 3, 15), effective_end_date=datetime.date(2023, 3, 16)
step=('months', 1)


[(datetime.date(2023, 1, 15), datetime.date(2023, 1, 31)),
 (datetime.date(2023, 2, 1), datetime.date(2023, 2, 28)),
 (datetime.date(2023, 3, 1), datetime.date(2023, 3, 15))]

In [20]:
# Calendar Quarter Test 1
sd = datetime.date(2023, 1, 1)
ed = datetime.date(2023, 9, 30)
ForecastAndBudgetPeriod(sd, ed, "Calendar Quarter").get_dates(inclusive=True)

start_date=datetime.date(2023, 1, 1)
end_date=datetime.date(2023, 9, 30), effective_end_date=datetime.date(2023, 10, 1)
step=('months', 3)


[(datetime.date(2023, 1, 1), datetime.date(2023, 3, 31)),
 (datetime.date(2023, 4, 1), datetime.date(2023, 6, 30)),
 (datetime.date(2023, 7, 1), datetime.date(2023, 9, 30))]

In [21]:
# Calendar Quarter Test 2
sd = datetime.date(2023, 1, 10)
ed = datetime.date(2023, 8, 10)
ForecastAndBudgetPeriod(sd, ed, "Calendar Quarter").get_dates(inclusive=True)

start_date=datetime.date(2023, 1, 10)
end_date=datetime.date(2023, 8, 10), effective_end_date=datetime.date(2023, 8, 11)
step=('months', 3)


[(datetime.date(2023, 1, 10), datetime.date(2023, 3, 31)),
 (datetime.date(2023, 4, 1), datetime.date(2023, 6, 30)),
 (datetime.date(2023, 7, 1), datetime.date(2023, 8, 10))]

In [22]:
# Calendar Quarter Test 3
sd = datetime.date(2023, 11, 1)
ed = datetime.date(2024, 4, 30)
ForecastAndBudgetPeriod(sd, ed, "Calendar Quarter").get_dates(inclusive=True)

start_date=datetime.date(2023, 11, 1)
end_date=datetime.date(2024, 4, 30), effective_end_date=datetime.date(2024, 5, 1)
step=('months', 3)


[(datetime.date(2023, 11, 1), datetime.date(2024, 1, 31)),
 (datetime.date(2024, 2, 1), datetime.date(2024, 4, 30))]

**Calendar Year and Annually**

- Calendar Year (assumes Jan-Dec FY, with stub): 6/30/23-7/1/25 (inclusive=False) => `[(6/30/23, 12/31/23), (1/1/24, 12/31/24), (1/1/25, 6/30/25)]`
- Annually (non calendar FY ending 10/31): 11/1/21-10/31/23 (inclusive=True) => `[(11/1/21, 10/31/22), (11/1/22, 10/31/23)]`

In [23]:
# Calendar Year Test
sd = datetime.date(2023, 6, 30)
ed = datetime.date(2025, 7, 1)
ForecastAndBudgetPeriod(sd, ed, "Calendar Year").get_dates(inclusive=False)

start_date=datetime.date(2023, 6, 30)
end_date=datetime.date(2025, 7, 1), effective_end_date=datetime.date(2025, 7, 1)
step=('years-cy', 1)


[(datetime.date(2023, 6, 30), datetime.date(2023, 12, 31)),
 (datetime.date(2024, 1, 1), datetime.date(2024, 12, 31)),
 (datetime.date(2025, 1, 1), datetime.date(2025, 6, 30))]

In [24]:
# Annually Test
sd = datetime.date(2021, 11, 1)
ed = datetime.date(2023, 10, 31)
ForecastAndBudgetPeriod(sd, ed, "Annually").get_dates(inclusive=True)

start_date=datetime.date(2021, 11, 1)
end_date=datetime.date(2023, 10, 31), effective_end_date=datetime.date(2023, 11, 1)
step=('years', 1)


[(datetime.date(2021, 11, 1), datetime.date(2022, 10, 31)),
 (datetime.date(2022, 11, 1), datetime.date(2023, 10, 31))]

**Edge Cases**

- End date is before start date: Calendar Quarter 11/1/23-4/30/23 (inclusive=True) => []
- End date would be the first day of a new period but is inclusive: Calendar Year 1/1/23-1/1/25 (inclusive=True) => `[(1/1/23), (12/31/23), (1/1/24, 12/31/24), (1/1/25, 1/1/25)]`
- TODO: missing start_date/missing end_date => error
- TODO: missing periodicity => weekly

In [25]:
# Edge Cases Test 1
sd = datetime.date(2023, 11, 1)
ed = datetime.date(2023, 4, 30)
ForecastAndBudgetPeriod(sd, ed, "Calendar Quarter").get_dates(inclusive=True)

start_date=datetime.date(2023, 11, 1)
end_date=datetime.date(2023, 4, 30), effective_end_date=datetime.date(2023, 5, 1)
step=('months', 3)


[]

In [26]:
# Edge Case Test 2
sd = datetime.date(2023, 1, 1)
ed = datetime.date(2025, 1, 1)
ForecastAndBudgetPeriod(sd, ed, "Calendar Year").get_dates(inclusive=True)

start_date=datetime.date(2023, 1, 1)
end_date=datetime.date(2025, 1, 1), effective_end_date=datetime.date(2025, 1, 2)
step=('years-cy', 1)


[(datetime.date(2023, 1, 1), datetime.date(2023, 12, 31)),
 (datetime.date(2024, 1, 1), datetime.date(2024, 12, 31)),
 (datetime.date(2025, 1, 1), datetime.date(2025, 1, 1))]